In [1]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import BertTokenizer
from processing import Processing
from classifier import CNN
from processing import Processing
from torch.utils.data import Dataset, DataLoader

In [32]:
import torch
import torch.nn as nn
import math
import torch.optim as optim
from processing import Processing
from torch.utils.data import Dataset, DataLoader

embed_len = 128
max_tokens = 50
# source: https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-conv1d-for-text-classification

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.data = Processing()
        self.data.process_all()
        self.embedding_layer = nn.Embedding(num_embeddings=len(self.data.vocab), embedding_dim=128)
        self.conv1 = nn.Conv1d(128, 32, kernel_size=7, padding="same")
        self.linear = nn.Linear(32, 5)

    def forward(self, X_batch):
        x = self.embedding_layer(X_batch)
        # Transpose the tensor to shape [16, 128, 930]
        x = x.transpose(1, 2)
        x = nn.functional.relu(self.conv1(x))
        x, _ = x.max(dim=-1)
        x = self.linear(x)
        return x

In [34]:

from sklearn.metrics import accuracy_score
class DatasetMaper(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
      
    def __len__(self):
        return len(self.x)
      
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


In [35]:
# Model      
model = CNN()
# Opmization function
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Loss function
loss_fn = nn.CrossEntropyLoss()
# Data
data = model.data

# Initialize dataset maper
train = DatasetMaper(data.x_train, data.y_train)
test = DatasetMaper(data.x_test, data.y_test)

# Initialize loaders
loader_train = DataLoader(train, batch_size=16)
loader_test = DataLoader(test, batch_size=16)

epochs = 3
learning_rate = 1e-3

for i in range(1, epochs+1):
    losses = []
    for X, Y in loader_train:
        Y_preds = model(X)
        loss = loss_fn(Y_preds, Y)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
    
    CalcValLossAndAccuracy(model, loss_fn, loader_test)




6115
Train Loss : 1.348
Valid Loss : 1.100
Valid Acc  : 0.579
Train Loss : 0.914
Valid Loss : 0.919
Valid Acc  : 0.669
Train Loss : 0.649
Valid Loss : 0.843
Valid Acc  : 0.712
